# 3D pose extraction from 2D orthographic projection with DRaM

In this notebook we will demonstrate how to use the determinant ratio matrix (DRaM) to determine the pose of a 3D point cloud from its 2D orthographic projection.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [47]:
import torch
import numpy as np
import matplotlib.pyplot as plt


In [48]:
from mpl_toolkits import mplot3d

In [49]:
from AdjQuat import utils
from AdjQuat import simulators
from AdjQuat import solutions

## 1. Data generation

First we will set up our data generation process.

In [50]:
# We define a manual seed for reproducilibity.
torch.manual_seed(0)

In [51]:
# First we generate random points in 3D space
npts = 50
xyz = torch.randn(npts, 3, dtype=torch.double)

In [52]:
xyz[0] # just to check our first point is the same for different runs

tensor([-2.3104, -0.3733, -1.0608], dtype=torch.float64)

We rotate this using the standard rotation matrix in terms of our quaternion ($\vec{q}=[q_0,q_1,q_2,q_3]$):

$$ {R}(\vec{q}) = \left[ \begin{array}{cc}  q_0^2 + q_1^2 - q_2^2 - q_3^2 & 2q_1q_2 - 2q_0q_3 & 2q_1q_3 + 2q_0q_2 \\ 2q_1q_2 + 2q_0q_3 & q_0^2 - q_1^2 + q_2^2 - q_3^2 & 2q_2q_3 - 2q_0q_1 \\ 2q_1q_3 - 2q_0q_2 & 2q_2q_3 + 2q_0q_1 &  q_0^2 - q_1^2 - q_2^2 + q_3^2 \end{array} \right]
$$

We will rotate this set of initial points by a small rotation to confirm our functions work as expected.

In [53]:
quaternion_small = np.array([np.cos(0.1),(1/np.sqrt(3))*np.sin(0.1),(1/np.sqrt(3))*np.sin(0.1),(1/np.sqrt(3))*np.sin(0.1)])

In [54]:
#convert to a rotation matrix and dot with our point cloud
rotation_small = utils.quat_to_rot(quaternion_small)
small_rot_xyz = rotation_small.dot(np.asarray(xyz).T).T

In [55]:
#convert to a projection matrix and dot with our point cloud
proj_small = utils.quat_to_proj(quaternion_small)
small_proj_xyz = proj_small.dot(np.asarray(xyz).T).T

In [56]:
fig = plt.figure();
ax = plt.axes(projection='3d');
ax.scatter3D(xyz[:,0],xyz[:,1],xyz[:,2],color='black',label='original');
ax.scatter3D(small_rot_xyz[:,0],small_rot_xyz[:,1],small_rot_xyz[:,2],color='cyan',label='rotated');
ax.scatter3D(small_proj_xyz[:,0],small_proj_xyz[:,1],np.zeros(len(small_proj_xyz.T[0]))-3,color='magenta',label='rotated & projected');
# ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
# ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
# ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(False)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

The function we will actually use to generate data also includes added error:

In [57]:
sigma = 0.1
UV, xyz_error = utils.make_U3(xyz,quaternion_small,sigma)

In [58]:
fig = plt.figure();
ax = plt.axes(projection='3d');
ax.scatter3D(xyz[:,0],xyz[:,1],xyz[:,2],color='black',label='XYZ');
ax.scatter3D(xyz_error[:,0],xyz_error[:,1],xyz_error[:,2],color='0.7',label='XYZ + $\sigma$');
#ax.scatter3D(UV[:,0],U[:,1],np.zeros(len(U.T[0]))-3,color='magenta',label='UV = (X + $\sigma$) $\cdot$ P');
# ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
# ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
# ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(False)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.legend()
plt.show()

<IPython.core.display.Javascript object>

### 2. Now let's evaluate our DRaM solution for this dataset.

Now we want to test equation (XX), which only gives a rotation matrix in the no noise case:

%% Derived  near bottom of. Pose-3D-working-1.nb WWW  - now reduce to determinants:
%%. detRulePose3D and revDetRulePose3D. and VVV do Rot3 cross-product det6, 9, 10
%%. TABLE of the remaining determinants in Pose-3D2D-Summary.nb. "DET TABLE"

$$
\tilde{R}(x,y,z;u,v)  = \left[ \begin{array}{ccc}
 \displaystyle\frac{d_{7}}{d_{1}} &     
              -\displaystyle\frac{d_{4}}{d_{1}} & 
                        \displaystyle\frac{d_{2}}{d_{1}}\\[0.15in]
 \displaystyle\frac{d_{8}}{d_{1}} &
    -\displaystyle\frac{d_{5}}{d_{1}} &
             \displaystyle\frac{d_{3}}{d_{1}}\\[0.15in]
         \displaystyle\frac{d_{6}}{d_{1}} &
       \displaystyle\frac{d_{9}}{d_{1}} &
          \displaystyle\frac{d_{10}}{d_{1}}\\ 
          \end{array} \right]
$$

where
$$\begin{equation}
\begin{aligned}
   & \ \  d_{1}\to \left[
\begin{array}{ccc}
 \text{xx} & \text{xy} & \text{xz} \\
 \text{xy} & \text{yy} & \text{yz} \\
 \text{xz} & \text{yz} & \text{zz} \\
\end{array} \right]     \\ 
d_{2}\to \left[ \begin{array}{ccc}
 \text{xx} & \text{xy} & \text{ux} \\
 \text{xy} & \text{yy} & \text{uy} \\
 \text{xz} & \text{yz} & \text{uz} \\
\end{array} \right] &  \ \ 
d_{3} \to \left[ \begin{array}{ccc}
 \text{xx} & \text{xy} & \text{vx} \\
 \text{xy} & \text{yy} & \text{vy} \\
 \text{xz} & \text{yz} & \text{vz} \\
\end{array} \right] &    
d_{4}\to \left[ \begin{array}{ccc}
 \text{xx} & \text{xz} & \text{ux} \\
 \text{xy} & \text{yz} & \text{uy} \\
 \text{xz} & \text{zz} & \text{uz} \\
\end{array} \right] \ \ \   \\ 
d_{5}\to \left[ \begin{array}{ccc}
 \text{xx} & \text{xz} & \text{vx} \\
 \text{xy} & \text{yz} & \text{vy} \\
 \text{xz} & \text{zz} & \text{vz} \\
\end{array} \right] &  \ \ 
d_{6}\to \left[ \begin{array}{ccc}
 \text{xx} & \text{ux} & \text{vx} \\
 \text{xy} & \text{uy} & \text{vy} \\
 \text{xz} & \text{uz} & \text{vz} \\
\end{array} \right]      &
 d_{7} \to \left[ \begin{array}{ccc}
 \text{xy} & \text{xz} & \text{ux} \\
 \text{yy} & \text{yz} & \text{uy} \\
 \text{yz} & \text{zz} & \text{uz} \\
\end{array} \right] \ \ \  \\ 
 d_{8} \to \left[ \begin{array}{ccc}
 \text{xy} & \text{xz} & \text{vx} \\
 \text{yy} & \text{yz} & \text{vy} \\
 \text{yz} & \text{zz} & \text{vz} \\
\end{array} \right] & \ \
d_{9}\to \left[ \begin{array}{ccc}
 \text{xy} & \text{ux} & \text{vx} \\
 \text{yy} & \text{uy} & \text{vy} \\
 \text{yz} & \text{uz} & \text{vz} \\
\end{array} \right]  
&   d_{10}\to \left[  \begin{array}{ccc}
 \text{xz} & \text{ux} & \text{vx} \\
 \text{yz} & \text{uy} & \text{vy} \\
 \text{zz} & \text{uz} & \text{vz} \\
\end{array} \right] .    \\
    \end{aligned}
  \end{equation}$$

In [ ]:
DRaM = solutions.make_R_tilde(xyz,UV)
DRaM

In [ ]:
#How close are we?
rotation_small

Looks close, but let's use least squares to evaluate our loss quantitatively.

In [ ]:
rot_mat_lsq = utils.least_squares_4_3D(rotation_small,xyz,UV)
rot_mat_lsq

In [ ]:
DRaM_lsq = utils.least_squares_4_3D(DRaM,xyz,UV)
DRaM_lsq

Because of the noise, the DRaM can actually produce a better solution than the original rotation we used to generate UV!

In [ ]:
#Just to check the no noise case:
UV_no_error, xyz_error_no_error = utils.make_U3(xyz,quaternion_small,0.0)
DRaM_no_error = solutions.make_R_tilde(xyz,UV_no_error)

In [ ]:
utils.least_squares_4_3D(rotation_small,xyz,UV_no_error)

In [ ]:
utils.least_squares_4_3D(DRaM_no_error,xyz,UV_no_error)

However (also because of the noise), this DRaM solution is not actually an orthogonal rotation matrix, so we need to correct using the Bar-Itzhack formalism.

In [ ]:
R_BI = solutions.make_M_opt_rot(xyz,UV)
R_BI # or DRaM_corrected

In [ ]:
#How close are we?
rotation_small

In [ ]:
DRaM_corr_lsq = utils.least_squares_4_3D(R_BI,xyz,UV)
DRaM_corr_lsq

This loss is higher than for the original DRaM, but it is a valid rotation matrix, and it is still lower than the original rotation used to generate the noisey data!

## 3. Compare to argmin result.

In [ ]:
xyz = np.asarray(xyz)
U = UV

In [ ]:
quat_argmin = utils.argmin_f(xyz, U)
quat_argmin

In [ ]:
quaternion_small

In [ ]:
rotation_argmin = utils.quat_to_rot(quat_argmin)

In [ ]:
argmin_lsq = utils.least_squares_4_3D(rotation_argmin,xyz,UV)
argmin_lsq

In [ ]:
#Let's plot these errors to compare
# %pylab inline
compare_errors = [rot_mat_lsq,DRaM_lsq,DRaM_corr_lsq,argmin_lsq]

In [ ]:
plt.figure(figsize=(4,3))
plt.plot([1,2,3,4],compare_errors ,'ko');
ticks = [1,2,3,4]
labels = ["rot_mat", "DRaM","DRaM_corr", "argmin"]
plt.xticks(ticks, labels)
plt.xlim((0.5,4.5))
plt.ylabel('lsq')

In [ ]:
#Just to check the no noise case for arg_min:
U = UV_no_error
quat_argmin_no_error = utils.argmin_f(xyz, U)
quat_argmin_no_error

In [ ]:
rotation_argmin_no_error = utils.quat_to_rot(quat_argmin_no_error)

In [ ]:
utils.least_squares_4_3D(rotation_argmin_no_error,xyz,UV_no_error)

Back to the errorful case - let's plot the xyz points we get if we use the estimated rotations.

In [ ]:
#Let's look at the result.
small_rot_error_xyz = rotation_small.dot(np.asarray(xyz_error).T).T
DRaM_xyz = DRaM.dot(xyz.T).T
DRaM_corr_xyz = R_BI.dot(xyz.T).T
argmin_xyz = rotation_argmin.dot(xyz.T).T

In [ ]:
# %matplotlib notebook

In [ ]:
fig = plt.figure();
ax = plt.axes(projection='3d');
ax.scatter3D(xyz[:,0],xyz[:,1],xyz[:,2], facecolors='none', edgecolors='k',linewidths=1.5,label='XYZ');
ax.scatter3D(small_rot_error_xyz[:,0],small_rot_error_xyz[:,1],small_rot_error_xyz[:,2],color='0.7',label='$R_{init}$ $\cdot$ (XYZ + $\sigma$)');
ax.scatter3D(small_rot_xyz[:,0],small_rot_xyz[:,1],small_rot_xyz[:,2],color='cyan',label='$R_{init} \cdot$ XYZ (lsq=%.2f)'%rot_mat_lsq);
ax.scatter3D(DRaM_corr_xyz[:,0],DRaM_corr_xyz[:,1],DRaM_corr_xyz[:,2],color='blue',label='$DRaM_{corr}$ (lsq=%.2f)'%DRaM_corr_lsq);
#ax.scatter3D(argmin_xyz[:,0],argmin_xyz[:,1],argmin_xyz[:,2],color='red',label=r'$R_{argmin}$ (lsq=%.2f)'%argmin_lsq);

ax.scatter3D(small_proj_xyz[:,0],small_proj_xyz[:,1],np.zeros(len(small_proj_xyz.T[0]))-3,color='magenta',label='UV');
# ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
# ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
# ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(False)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.legend()
plt.show()

In [ ]:
fig = plt.figure();
ax = plt.axes(projection='3d');
ax.scatter3D(xyz[:,0],xyz[:,1],xyz[:,2], facecolors='none', edgecolors='k',linewidths=1.5,label='XYZ');
ax.scatter3D(small_rot_error_xyz[:,0],small_rot_error_xyz[:,1],small_rot_error_xyz[:,2],color='0.7',label='$R_{init}$ $\cdot$ (XYZ + $\sigma$)');
#ax.scatter3D(small_rot_xyz[:,0],small_rot_xyz[:,1],small_rot_xyz[:,2],color='cyan',label='$R_{init} \cdot$ XYZ (lsq=%.2f)'%rot_mat_lsq);
ax.scatter3D(DRaM_corr_xyz[:,0],DRaM_corr_xyz[:,1],DRaM_corr_xyz[:,2],color='blue',label='$DRaM_{corr}$ (lsq=%.2f)'%DRaM_corr_lsq);
#ax.scatter3D(argmin_xyz[:,0],argmin_xyz[:,1],argmin_xyz[:,2],color='red',label=r'$R_{argmin}$ (lsq=%.2f)'%argmin_lsq);

for i in range(len(xyz[:,0])):
    ax.plot([xyz[i,0],small_rot_error_xyz[i,0]],[xyz[i,1],small_rot_error_xyz[i,1]],[xyz[i,2],small_rot_error_xyz[i,2]],color='black',linestyle='--',alpha=0.4);

ax.scatter3D(small_proj_xyz[:,0],small_proj_xyz[:,1],np.zeros(len(small_proj_xyz.T[0]))-3,color='magenta',label='UV');
# ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
# ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
# ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.grid(False)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.legend()
plt.show()
#still to do: draw chord

### 4. Testing DRaM_corrected with many replicates and various parameters.

In [ ]:
R_set, xyz_set, U_set = simulators.generate_data_3D(100,50,0.1) #100 replicates, npts=50, sigma=0.1

In [ ]:
R_set

In [ ]:
Rt_estimated = [solutions.make_R_tilde(xyz, U) for xyz, U in zip(xyz_set, U_set)]
lsq = [utils.least_squares_4_3D(Rt_estimated[idx],xyz, U) for idx, (xyz, U) in enumerate(zip(xyz_set, U_set ))]
lsq_rot = [utils.least_squares_4_3D(R,xyz, U) for (R, xyz, U) in zip(R_set, xyz_set, U_set )]
Mq_estimated = [solutions.make_M_opt_rot(xyz,U) for xyz, U in zip(xyz_set, U_set)]
lsq_M = [utils.least_squares_4_3D(Mq_estimated[idx],xyz,U) for idx, (xyz, U) in enumerate(zip(xyz_set, U_set ))]

In [ ]:
#sorting indices to make it more interprettable
lsq_M_sorted_indices = np.argsort(lsq_M)

In [ ]:
lsq_M_sortedbyM = [lsq_M[i] for i in lsq_M_sorted_indices]
lsq_sortedbyM  = [lsq[i] for i in lsq_M_sorted_indices]
lsq_rot_sortedbyM  = [lsq_rot[i] for i in lsq_M_sorted_indices]

In [ ]:
q_argmin = [utils.argmin_f(xyz, U) for xyz, U in zip(xyz_set, U_set)]
R_argmin = [utils.quat_to_rot(q) for q in q_argmin]
lsq_argmin = [utils.least_squares_4_3D(R_argmin[idx],xyz,U) for idx, (xyz, U) in enumerate(zip(xyz_set, U_set ))]


In [ ]:
lsq_argmin_sortedbyM  = [lsq_argmin[i] for i in lsq_M_sorted_indices]

In [ ]:
diff = [lsq_M[i]-lsq_argmin[i] for i in lsq_M_sorted_indices]
diff_20 = [20*(lsq_M[i]-lsq_argmin[i])+0.55 for i in lsq_M_sorted_indices]

In [ ]:
fig,ax = plt.subplots(figsize=[8,3])
plt.plot(lsq_rot_sortedbyM,'c.',label=r'$R_{init}$');
plt.plot(lsq_argmin_sortedbyM,'r.',label=r'$R_{argmin}$');
plt.plot(lsq_M_sortedbyM,'b.',label=r'$DRaM_{corrected}$');
plt.plot(lsq_sortedbyM,'k.',label=r'$DRaM(x,y,z;u,v)$');
plt.axhline(y=np.mean(lsq_M),linestyle='--',color='b',alpha=0.3)
plt.axhline(y=np.mean(lsq_rot),linestyle='--',color='c',alpha=0.3)
plt.axhline(y=np.mean(lsq),linestyle='--',color='k',alpha=0.3)
plt.legend()
plt.ylabel('lsq',fontsize=12,fontweight='bold')
plt.xlabel(r'index sorted by $DRaM_{corrected}$',fontsize=12,fontweight='bold')

ax2=ax.twinx()
ax2.plot(diff,'g.')
ax2.plot(diff,'g.',linestyle='--',alpha=0.5,label='diff')
plt.ylim((0,0.1))
plt.legend(bbox_to_anchor=(1, 0.3))
plt.ylabel(r'$DRaM_{corrected} - R_{argmin}$',fontsize=11,fontweight='bold')

plt.title('sigma=0.1, N=50');
plt.tight_layout();

plt.savefig('DRaM_orthographic.png',dpi=300)
#plt.savefig('DRaM_orthographic.eps',dpi=300)